이 코드는 한국어 텍스트 데이터를 기반으로 여러 종류의 평가 지표(회귀 및 분류)를 예측하기 위한 딥러닝 모델을 구축하고, 훈련시키며, 평가하는 데 사용됩니다. 구체적인 내용은 다음과 같습니다:

1. **모델 정의**: 
   - `KoBERTRubricScorer`: 한국어 BERT(`KoBERT`) 모델을 사용하여 회귀 작업을 수행하는 모델입니다. 여러 회귀 작업(점수)을 예측합니다.
   - `KoBERTRubricClassifier`: `KoBERT`를 사용하여 분류 작업을 수행하는 모델입니다. 다양한 클래스를 분류합니다.
   - `KoBERTRubricMTLScorer`: 다중 작업 학습(Multi-Task Learning)을 위한 모델로, 회귀 및 분류 작업을 동시에 수행합니다.
   - `KoBERTRubricMTLRegressor`: 다중 회귀 작업을 수행하는 모델로, 여러 회귀 출력을 동시에 예측합니다.

2. **데이터셋 및 데이터 로더**:
   - `RubricScoringDataset`: 텍스트 데이터셋을 전처리하고, 토크나이징하여 모델이 사용할 수 있는 형식으로 변환합니다. 필요한 경우 데이터 증강을 수행합니다.
   - `create_data_loader`: 데이터셋을 배치(batch) 단위로 로드하고, 훈련 및 검증/테스트를 위해 셔플 옵션을 관리합니다.

3. **훈련 및 검증 로직**:
   - `RubricScorerTrainer`: 모델을 훈련시키고 검증하는 클래스입니다. 각 에폭 동안 훈련 손실과 정확도를 계산하고, 검증 데이터셋에 대해 성능을 평가합니다.

4. **앙상블**:
   - `RubricScorerEnsemble`: 훈련된 여러 모델의 예측 결과를 앙상블하여 최종 예측 결과를 생성합니다. 여기서는 단순 평균(average) 방식을 사용합니다.

5. **메인 함수 (`main`)**:
   - 데이터셋을 로드하고 전처리합니다.
   - 정의된 모델을 초기화하고 훈련시킵니다.
   - 훈련된 모델을 앙상블하여 테스트 데이터에 대한 예측을 수행합니다.
   - 평가 함수를 사용하여 모델 성능을 측정하고 결과를 출력합니다.

전체적으로 이 코드는 딥러닝 기반의 다중 작업 학습 모델을 구축하고 평가하기 위한 완전한 파이프라인을 제공합니다. 데이터 전처리부터 모델 훈련, 앙상블, 평가에 이르기까지 전체적인 딥러닝 작업 흐름을 다룹니다.

```json
[
    {
        "prompt": "인공지능의 발전이 사회에 미치는 영향에 대해 논하시오.",
        "rubric": [
            {"name": "내용의 적절성", "description": "주제와 관련된 내용을 적절히 다루었는가?", "max_score": 5},
            {"name": "논리적 구조", "description": "글의 구조가 논리적으로 잘 조직되었는가?", "max_score": 5},
            {"name": "어휘의 다양성", "description": "다양하고 적절한 어휘를 사용하였는가?", "max_score": 3},
            {"name": "문법 및 맞춤법", "description": "문법 및 맞춤법이 정확한가?", "max_score": 2}
        ],
        "response": "인공지능의 발전은 사회에 많은 변화를 가져올 것입니다. ...",
        "scores": [3, 4, 2, 1]
    },
    ...
]
```

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertConfig
from torch.utils.data import DataLoader
from konlpy.tag import Okt
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
import random

In [2]:
# 모델 정의
class KoBERTRubricScorer(nn.Module):
    def __init__(self, num_tasks, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        config.num_labels = num_tasks
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.regressor = nn.Linear(hidden_size, num_tasks)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

class KoBERTRubricClassifier(nn.Module):
    def __init__(self, num_classes, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        config.num_labels = num_classes
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

class KoBERTRubricMTLScorer(nn.Module):
    def __init__(self, num_tasks, num_classes, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.regression_heads = nn.ModuleList([nn.Linear(hidden_size, 1) for _ in range(num_tasks)])
        self.classification_head = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        regression_outputs = [head(pooled_output) for head in self.regression_heads]
        regression_outputs = torch.cat(regression_outputs, dim=-1)
        classification_output = self.classification_head(pooled_output)
        return regression_outputs, classification_output

class KoBERTRubricMTLRegressor(nn.Module):
    def __init__(self, num_tasks, hidden_size=768, dropout_rate=0.1):
        super().__init__()
        config = BertConfig.from_pretrained('monologg/kobert')
        self.bert = BertModel.from_pretrained('monologg/kobert', config=config)
        self.dropout = nn.Dropout(dropout_rate)
        self.regression_heads = nn.ModuleList([nn.Linear(hidden_size, 1) for _ in range(num_tasks)])

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        regression_outputs = [head(pooled_output) for head in self.regression_heads]
        regression_outputs = torch.cat(regression_outputs, dim=-1)
        return regression_outputs

In [3]:
# 데이터셋 정의
class RubricScoringDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length, okt=None, stopwords=None, synonym_dict=None, augment_ratio=0.0):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.okt = okt
        self.stopwords = stopwords
        self.synonym_dict = synonym_dict
        self.augment_ratio = augment_ratio

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        prompt = item['prompt']
        response = item['response']
        rubric_text = ' '.join([f"{rubric['name']}: {rubric['description']}" for rubric in item['rubric']])
        max_scores = torch.tensor([rubric['max_score'] for rubric in item['rubric']], dtype=torch.float32)

        if self.okt is not None and self.stopwords is not None:
            tokens = self.okt.morphs(response)
            tokens = [token for token in tokens if token not in self.stopwords]
            response = ' '.join(tokens)
        if self.augment_ratio > 0 and random.random() < self.augment_ratio:
            response = self.augment_text(response)

        text = f"{prompt} {response} {rubric_text}"
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        labels = torch.tensor(item['scores'], dtype=torch.float32)
        return inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze(), labels, max_scores

    def augment_text(self, text):
        if self.synonym_dict is not None:
            words = text.split()
            for i, word in enumerate(words):
                if word in self.synonym_dict and random.random() < 0.1:
                    words[i] = random.choice(self.synonym_dict[word])
            text = ' '.join(words)
        return text

In [4]:
# 훈련 및 검증 트레이너
class RubricScorerTrainer:
    def __init__(self, model, train_dataloader, val_dataloader, optimizer, criterion, device):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device

    def train(self, num_epochs):
        self.model.to(self.device)
        for epoch in range(num_epochs):
            self.model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            for batch in self.train_dataloader:
                input_ids, attention_mask, labels = [data.to(self.device) for data in batch]
                self.optimizer.zero_grad()
                outputs = self.model(input_ids, attention_mask)
                # 예측된 점수를 max_scores로 제한하는 로직을 추가할 수 있음
                # 예: outputs = torch.min(outputs, max_scores)

                loss = self.compute_loss(outputs, labels)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
                train_correct += self.compute_accuracy(outputs, labels)
                train_total += labels.size(0)

            train_loss /= len(self.train_dataloader)
            train_accuracy = train_correct / train_total if train_total > 0 else None
            val_loss, val_accuracy = self.validate()
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy if train_accuracy else '-'}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy if val_accuracy else '-'}")

    def compute_loss(self, outputs, labels):
        if isinstance(self.model, (KoBERTRubricClassifier, KoBERTRubricMTLScorer)):
            return self.criterion(outputs[-1], labels.long())  # Assumes classification labels are at the end
        else:
            return self.criterion(outputs, labels)

    def compute_accuracy(self, outputs, labels):
        if isinstance(self.model, (KoBERTRubricClassifier, KoBERTRubricMTLScorer)):
            _, predicted = torch.max(outputs[-1], 1)
            return (predicted == labels.long()).sum().item()
        return 0  # No accuracy for regression tasks

    def validate(self):
        self.model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in self.val_dataloader:
                input_ids, attention_mask, labels = [data.to(self.device) for data in batch]
                outputs = self.model(input_ids, attention_mask)
                val_loss += self.compute_loss(outputs, labels).item()
                val_correct += self.compute_accuracy(outputs, labels)
                val_total += labels.size(0)

        val_loss /= len(self.val_dataloader)
        val_accuracy = val_correct / val_total if val_total > 0 else None
        return val_loss, val_accuracy

In [6]:
# 앙상블 클래스 정의
class RubricScorerEnsemble:
    def __init__(self, models, device):
        self.models = models
        self.device = device

    def predict(self, dataloader):
        total_predictions = []
        with torch.no_grad():
            for model, _ in self.models:
                model.to(self.device)
                model.eval()
                predictions = []
                for batch in dataloader:
                    input_ids, attention_mask, _ = [data.to(self.device) for data in batch]
                    outputs = model(input_ids, attention_mask)
                    if isinstance(model, (KoBERTRubricClassifier, KoBERTRubricMTLScorer)):
                        predictions.append(torch.softmax(outputs[-1], dim=1).cpu().numpy())  # Classifier prediction
                    else:
                        predictions.append(outputs.cpu().numpy())  # Regressor prediction
                total_predictions.append(np.hstack(predictions))

        # Mean ensemble
        ensemble_predictions = np.mean(np.array(total_predictions), axis=0)
        return ensemble_predictions

In [5]:
def create_data_loader(dataset, batch_size, shuffle=True):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def evaluate_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, f1

def evaluate_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return mse

In [ ]:
def main():
    # 데이터 로드
    train_data = pd.read_csv('train_data.csv')
    val_data = pd.read_csv('val_data.csv')
    test_data = pd.read_csv('test_data.csv')

    # 토크나이저 및 전처리 도구 로드
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
    okt = Okt()
    stopwords = ['은', '는', '이', '가', '을', '를']
    synonym_dict = {'좋아하다': ['즐기다', '기뻐하다'], '훌륭하다': ['우수하다', '뛰어나다']}

    # 데이터셋 생성
    train_dataset = RubricScoringDataset(train_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords, synonym_dict=synonym_dict, augment_ratio=0.1)
    val_dataset = RubricScoringDataset(val_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords)
    test_dataset = RubricScoringDataset(test_data, tokenizer, max_length=128, okt=okt, stopwords=stopwords)

    # 데이터 로더 생성
    batch_size = 16
    train_dataloader = create_data_loader(train_dataset, batch_size, shuffle=True)
    val_dataloader = create_data_loader(val_dataset, batch_size, shuffle=False)
    test_dataloader = create_data_loader(test_dataset, batch_size, shuffle=False)

    # 디바이스 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 모델 생성 및 훈련
    models = {
        "Rubric Scorer": KoBERTRubricScorer(num_tasks=3),
        "Rubric Classifier": KoBERTRubricClassifier(num_classes=5),
        "MTL Rubric Scorer": KoBERTRubricMTLScorer(num_tasks=3, num_classes=5),
        "MTL Rubric Regressor": KoBERTRubricMTLRegressor(num_tasks=3)
    }

    trained_models = {}
    num_epochs = 10
    for name, model in models.items():
        if 'Classifier' in name:
            criterion = nn.CrossEntropyLoss()
        else:
            criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        trainer = RubricScorerTrainer(model, train_dataloader, val_dataloader, optimizer, criterion, device)
        print(f"Training {name}...")
        trainer.train(num_epochs)
        trained_models[name] = model

    # 앙상블 설정 및 예측
    ensemble = RubricScorerEnsemble(list(trained_models.values()), device)
    ensemble_predictions = ensemble.predict(test_dataloader)

    # 평가 및 결과 출력
    regression_labels = test_data[['score1', 'score2', 'score3']].values
    regression_mse = evaluate_regression(regression_labels, ensemble_predictions[:, :3])
    print(f"Ensemble Regression MSE: {regression_mse:.4f}")

    classification_labels = test_data['score4'].values  # 가정: score4는 분류 레이블
    classification_predictions = np.argmax(ensemble_predictions[:, 3:], axis=1)  # 가정: 마지막 5개 컬럼이 분류 결과
    accuracy, f1 = evaluate_classification(classification_labels, classification_predictions)
    print(f"Ensemble Classification Accuracy: {accuracy:.4f}")
    print(f"Ensemble Classification F1 Score: {f1:.4f}")

if __name__ == '__main__':
    main()
